# 1. 导入数据

In [1]:
import os
import tensorflow as tf # 导入 TF 库
from tensorflow import keras # 导入 TF 子库 keras
from tensorflow.keras import layers, optimizers, datasets # 导入 TF 子库等

In [9]:

(x, y), (x_val, y_val) = datasets.mnist.load_data() # 加载 MNIST 数据集
x = 2 * tf.convert_to_tensor(x, dtype=tf.float32)/255.-1 # 转换为浮点张量，并缩放到-1~1
y = tf.convert_to_tensor(y, dtype=tf.int32) # 转换为整形张量
# y = tf.one_hot(y, depth=10) # one-hot 编码
print(x.shape, y.shape)
train_dataset = tf.data.Dataset.from_tensor_slices((x, y)) # 构建数据集对象
train_dataset = train_dataset.batch(512) # 批量训练

(60000, 28, 28) (60000,)


# 2. 网络搭建
对于第一层模型来说，它接受的输入𝒙 ∈ 𝑅784，输出𝒉1 ∈ 𝑅256设计为长度为 256 的向
量，我们不需要显式地编写𝒉1 = ReLU(𝑾1𝒙 + 𝒃1)的计算逻辑，在 TensorFlow 中通过一行代码即可实现

In [3]:
from tensorflow.keras import layers
import tensorflow as tf
# 创建一层网络，设置输出节点数为 256，激活函数类型为 ReLU
layers.Dense(256, activation='relu')

使用 TensorFlow 的 Sequential 容器可以非常方便地搭建多层的网络。对于 3 层网络，我们
可以通过快速完成 3 层网络的搭建。

In [4]:
from tensorflow import keras

# 利用 Sequential 容器封装 3 个网络层，前网络层的输出默认作为下一层的输入
model = keras.Sequential([ # 3 个非线性层的嵌套模型
 layers.Dense(256, activation='relu'), # 隐藏层 1
 layers.Dense(128, activation='relu'), # 隐藏层 2
 layers.Dense(10)]) # 输出层，输出节点数为 10

第 1 层的输出节点数设计为 256，第 2 层设计为 128，输出层节点数设计为 10。直接调用
这个模型对象 model(x)就可以返回模型最后一层的输出𝑜。

# 3.模型训练

搭建完成 3 层神经网络的对象后，给定输入𝒙，调用 model(𝒙)得到模型输出𝑜后，通过MSE 损失函数计算当前的误差ℒ：

In [5]:
optimizer = optimizers.SGD(learning_rate=0.001)

In [10]:
with tf.GradientTape() as tape: # 构建梯度记录环境
    # 打平操作，[b, 28, 28] => [b, 784]
    x = tf.reshape(x, (-1, 28*28))
    # Step1. 得到模型输出 output [b, 784] => [b, 10]
    out = model(x)
    # [b] => [b, 10]
    y_onehot = tf.one_hot(y, depth=10)
    # 计算差的平方和，[b, 10]
    loss = tf.square(out-y_onehot)
    # 计算每个样本的平均误差，[b]
    loss = tf.reduce_sum(loss) / x.shape[0]
# Step3. 计算参数的梯度 w1, w2, w3, b1, b2, b3
grads = tape.gradient(loss, model.trainable_variables)
# w' = w - lr * grad，更新网络参数
optimizer.apply_gradients(zip(grads, model.trainable_variables))

<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=2>

再利用 TensorFlow 提供的自动求导函数 tape.gradient(loss, model.trainable_variables)求出模
型中所有参数的梯度信息𝜕ℒ/𝜕𝜃 , 𝜃 ∈ {𝑾1, 𝒃1,𝑾2, 𝒃2,𝑾3, 𝒃3}